In [9]:
import pandas as pd
import folium
from pyproj import Transformer

In [30]:
city = 'orleans'
stations = pd.read_csv('../data/' + city + '/stations/netatmo_stations.csv', delimiter=',')
stations.head()

,device_id,module_id,lon,lat,processed
0,70:ee:50:00:39:d8,02:00:00:63:0b:de,1.971010,47.827425,True
1,70:ee:50:33:2c:f2,02:00:00:33:1d:ca,1.971870,47.828334,True
2,70:ee:50:2b:61:b4,02:00:00:2b:61:c2,1.972491,47.834341,True
3,70:ee:50:2c:af:ae,02:ff:ff:2c:b9:26,1.970931,47.835205,True
4,70:ee:50:b4:fb:1c,02:00:00:b5:0f:c6,1.971673,47.834988,True


In [31]:
df = pd.read_csv('../data/' + city + '/stations/temperatures.csv', delimiter=',')
df.head()

,date,time,temp,station
0,2023-06-01,1,18.8,70:ee:50:00:0e:ac
1,2023-06-01,2,18.1,70:ee:50:00:0e:ac
2,2023-06-01,3,17.0,70:ee:50:00:0e:ac
3,2023-06-01,4,16.4,70:ee:50:00:0e:ac
4,2023-06-01,5,16.7,70:ee:50:00:0e:ac


In [32]:
mean_temp = df.groupby('station')['temp'].mean().reset_index()
mean_temp.columns = ['station', 'mean_temp']
mean_temp.head()

,station,mean_temp
0,70:ee:50:00:0e:ac,22.471234
1,70:ee:50:00:2f:1a,22.578598
2,70:ee:50:00:39:d8,21.066438
3,70:ee:50:00:64:08,21.647581
4,70:ee:50:00:66:52,20.382090


In [33]:
plot_df = mean_temp.merge(stations, left_on='station', right_on='device_id')
plot_df.head()

,station,mean_temp,device_id,module_id,lon,lat,processed
0,70:ee:50:00:0e:ac,22.471234,70:ee:50:00:0e:ac,02:00:00:00:13:ea,1.914010,47.929550,True
1,70:ee:50:00:2f:1a,22.578598,70:ee:50:00:2f:1a,02:00:00:00:28:ee,1.923675,47.927192,True
2,70:ee:50:00:39:d8,21.066438,70:ee:50:00:39:d8,02:00:00:63:0b:de,1.971010,47.827425,True
3,70:ee:50:00:64:08,21.647581,70:ee:50:00:64:08,02:00:00:00:6b:98,1.898878,47.905130,True
4,70:ee:50:00:66:52,20.382090,70:ee:50:00:66:52,02:00:00:00:67:3a,1.973378,47.905057,True


In [34]:
from branca.colormap import LinearColormap

colors = ["#3b4cc0", "#f7f7f7", "#b40426"] # blue → white → red
colormap = LinearColormap(
    colors=colors,
    vmin=plot_df.mean_temp.min(),
    vmax=plot_df.mean_temp.max(),
    caption="Mean temperature (°C)"
)

In [35]:
# Create a map
m = folium.Map(
    location=[plot_df.lat.mean(), plot_df.lon.mean()],
    zoom_start=13,
    control_scale=True)

# Add markers
for _, row in plot_df.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=8,
        popup=(f"Station: {row.station}<br>"
               f"Mean temp: {row.mean_temp:.2f} °C"),
        color=colormap(row.mean_temp),
        fill=True,
        fill_opacity=0.9
    ).add_to(m)

# Add a legend
colormap.caption = "Mean temperature (°C)"
colormap.add_to(m)

# Display
m

In [23]:
# Print stations with lowest temperatures
lowest = (mean_temp.sort_values('mean_temp').head(5))
print(lowest)

               station  mean_temp
33   70:ee:50:19:0b:32  16.994591
123  70:ee:50:a4:32:be  19.655914
54   70:ee:50:2c:b8:90  20.204302
38   70:ee:50:20:d6:4c  20.221307
58   70:ee:50:2e:e5:54  20.236207
